# Reducing Hallucinations from AI Agents using Long-Term Memory
### Introduction to Critique-Based Contexting with OpenAI, LangChain, and LanceDB
AI agents can help simplify and automate tedious workflows. By going through this notebook, we'll introduce how you can reduce hallucinations from AI agents by using critique-based contexting with a fitness trainer agent example.

### Installing dependencies

In [3]:
!pip install openai==0.28 langchain==0.0.354 google-search-results lancedb python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.3/803.3 kB 21.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.4/87.4 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.5/229.5 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.6/18.6 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 10.8 MB/s eta 0:00:00
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32003 sha256=4aa2ba1f551eb442521e7c712b95a862c1b9

### Importing libraries

In [4]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI

Now let's import our environment variables via `load_dotenv()`.

In [5]:
from dotenv import load_dotenv

load_dotenv()

False

We now specify and connect to the path `data/agent-lancedb` to store our vector database.

In [6]:
import lancedb

db = lancedb.connect("data/agent-lancedb")

To create embeddings out of the text, we'll call the OpenAI embeddings API (ada2 text embeddings model) to get embeddings.

In [22]:
import openai
import os

openai.api_key = "sk-"  # PASTE YOUR KEY HERE


def embed_func(c):
    rs = openai.Embedding.create(input=c, engine="text-embedding-ada-002")
    return [record["embedding"] for record in rs["data"]]

Now, we'll create a `LangChain` tool that allows our agent to insert critiques, which uses a `pydantic` schema to guide the agent on what kind of results to insert.

In LanceDB the primary abstraction you'll use to work with your data is a **Table**.  
A Table is designed to store large numbers of columns and huge quantities of data! For those interested, a LanceDB is columnar-based, and uses Lance, an open data format to store data.

This tool will create a Table if it does not exist and store the relevant information (the embedding, actions, and critiques).

### Considering the image below
1. ideation step gets a predefined number of output proposals (ideas) from the LLM.

2. critiques all of the ideas looking for possible flaws in the proposals and picking the most appropriate suggestion.

3. resolve, the LLM tries to improve the best idea from the [2] critique step. The output here constitutes the final answer.

![image](https://miro.medium.com/v2/resize:fit:1400/1*dMfLxIR-s8osxkVUbORtGg.png)

### Function to take input

In [23]:
from langchain.tools import tool
from pydantic import BaseModel, Field


class InsertCritiquesInput(BaseModel):
    info: str = Field(
        description="should be demographics or interests or other information about the exercise request provided by the user"
    )
    actions: str = Field(
        description="numbered list of langchain agent actions taken (searched for, gave this response, etc.)"
    )
    critique: str = Field(
        description="negative constructive feedback on the actions you took, limitations, potential biases, and more"
    )


@tool("insert_critiques", args_schema=InsertCritiquesInput)
def insert_critiques(info: str, actions: str, critique: str) -> str:
    "Insert actions and critiques for similar exercise requests in the future." ""
    table_name = "exercise-routine"
    if table_name not in db.table_names():
        tbl = db.create_table(
            table_name,
            [{"vector": embed_func(info)[0], "actions": actions, "critique": critique}],
        )
    else:
        tbl = db.open_table(table_name)
        tbl.add(
            [{"vector": embed_func(info)[0], "actions": actions, "critique": critique}]
        )
    return "Inserted and done."

Similarly, let's create a tool for retrieving critiques. We'll retrieve the actions and critiques from the top 5 most similar user inputs.

### Retrieve Input

In [24]:
class RetrieveCritiquesInput(BaseModel):
    query: str = Field(
        description="should be demographics or interests or other information about the exercise request provided by the user"
    )


@tool("retrieve_critiques", args_schema=RetrieveCritiquesInput)
def retrieve_critiques(query: str) -> str:
    "Retrieve actions and critiques for similar exercise requests." ""
    table_name = "exercise-routine"
    if table_name in db.table_names():
        tbl = db.open_table(table_name)
        results = (
            tbl.search(embed_func(query)[0])
            .limit(5)
            .select(["actions", "critique"])
            .to_pandas()
        )
        results_list = results.drop("vector", axis=1).values.tolist()
        return (
            "Continue with the list with relevant actions and critiques which are in the format [[action, critique], ...]:\n"
            + str(results_list)
        )
    else:
        return "No info, but continue."

Let's now use LangChain to load our tools in. This includes our custom tools as well as a Google Search tool that uses SerpApi. We will use OpenAI's `gpt-3.5-turbo-0613` as our LLM.
Note: You need to sign up on serpapi and paste the API key here

In [25]:
os.environ["SERPAPI_API_KEY"] = ".."

llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")
tools = load_tools(["serpapi"], llm=llm)
tools.extend([insert_critiques, retrieve_critiques])

Before we run our agent, let's create a function that defines our prompt that we pass in to the agent, which allows us to pass in client information.

### Create Prompt Function

In [26]:
def create_prompt(info: str) -> str:
    prompt_start = (
        "Please execute actions as a fitness trainer based on the information about the user and their interests below.\n\n"
        + "Info from the user:\n\n"
    )
    prompt_end = (
        "\n\n1. Retrieve using user info and review the past actions and critiques if there is any\n"
        + "2. Keep past actions and critiques in mind while researching for an exercise routine with steps which we respond to the user\n"
        + "3. Before returning the response, it is of upmost importance to insert the actions you took (numbered list: searched for, found this, etc.) and critiques (negative feedback: limitations, potential biases, and more) into the database for getting better exercise routines in the future. \n"
    )
    return prompt_start + info + prompt_end

### Run Agent

Finally, let's create our run_agent function. We'll use the `STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION` agent in order to allow us to use multi-input tools (since we need to add client input, actions, and critiques as arguments).

In [27]:
def run_agent(info):
    agent = initialize_agent(
        tools,
        llm,
        agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True,
    )
    agent.run(input=create_prompt(info))

Let's run. Feel free to use your own input!

Notice that in the first run there wouldn't be any critiques yet, since the database is empty. After the first run, critiques should appear. The provided output is the result of a particular run after a few runs.

In [29]:
run_agent(
    "My name is Tevin, I'm a 19 year old university student at CMU. I love running."
)



> Entering new AgentExecutor chain...
Thought: I need to retrieve any past actions and critiques related to fitness training for users with similar demographics and interests. Then, I can use that information to research and provide an exercise routine for Tevin. Finally, I will insert the actions and critiques into the database for future reference.

Action:
```
{
  "action": "retrieve_critiques",
  "action_input": "19 year old university student, running"
}
```


Observation: No info, but continue.
Thought:I couldn't find any past actions and critiques related to fitness training for users with similar demographics and interests. I will proceed with researching and providing an exercise routine for Tevin.

Action:
```
{
  "action": "Search",
  "action_input": "exercise routine for running"
}
```
Observation: ['Said training involves six exercises: chest press, pull down, back extension, leg press, abdominal flexion, and either hip abduction or ...', 'These quick, effective 30-minut